# CNNで画像認識

## ライブラリの読み込み

In [35]:
# データ処理のライブラリ
import numpy as np 
import pandas as pd

# データ可視化のライブラリ
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

# 機械学習ライブラリ scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import fetch_mldata

# 機械学習ライブラリ keras
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

## データセットの読み込み

In [36]:
# MNISTデータセットの読み込み
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

# 訓練・テストデータの結合
X = np.append(X_train, X_test, axis=0)
X = X.reshape(X.shape[0], 784)
y = np.append(y_train, y_test, axis=0)

# train_test_splitを使い再分割
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=42) 

# 訓練データの最初の1万件のみへ縮小
X_train = X_train[:10000]
y_train = y_train[:10000]

# テストデータの最初の1万件のみへ縮小
X_test = X_test[:10000]
y_test = y_test[:10000]

# 正規化
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

## データの前処理

In [37]:
# 特徴量データのサイズ
print(X_train.shape)
print(X_test.shape)

(10000, 784)
(10000, 784)


In [38]:
# Keras用にデータを整形
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
print(X_train.shape)
print(X_test.shape)

(10000, 28, 28, 1)
(10000, 28, 28, 1)


In [39]:
# ターゲットデータの確認
print(y_train[0:5])
print(y_test[0:5])

[5 4 8 0 2]
[8 4 8 7 7]


In [40]:
# ターゲットデータをクラスベクトルへ変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print(y_train[0:5])
print(y_test[0:5])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


## 

In [41]:
# Sequentialモデル
model = Sequential()

In [42]:
# 入力層と畳み込み層
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))

In [43]:
# プール層
model.add(MaxPooling2D(pool_size=(2, 2)))

Dropoutは、意図的にニューラルネットワークのユニットをランダムに 0 とする。  
Dropoutにより、過学習の防止に役立つ。  

Dropoutの引数 0.25 は、ユニット全体の25%をランダムに 0 とすることを意味する。  
どのような割合にするのかは、CNNの構造・データの特性により様々。  
一般的には、0.25 or 0.5 が使用される。

In [44]:
# ドロップアウト
model.add(Dropout(0.25))

In [45]:
# 全結合層
# １つ前の層から出力されたデータを平らなベクトルにする
model.add(Flatten())
# 隠れ層を追加
model.add(Dense(128, activation='relu')) 

In [46]:
# 出力層
model.add(Dense(10, activation='softmax'))

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1352)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               173184    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 174,554
Trainable params: 174,554
Non-trainable params: 0
_________________________________________________________________


## モデルのコンパイル

In [48]:
# モデル コンパイル
# loss: 損失関数
# optimizer: 最適化アルゴリズム
# metrics: 評価関数
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

## モデルの訓練

In [49]:
# モデル訓練
model.fit(X_train, y_train, epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
10000/10000 [==============================] - 3s 265us/step - loss: 1.2720 - acc: 0.6289
Epoch 2/50
10000/10000 [==============================] - 2s 212us/step - loss: 0.5507 - acc: 0.8327
Epoch 3/50
10000/10000 [==============================] - 2s 213us/step - loss: 0.4532 - acc: 0.8606
Epoch 4/50
10000/10000 [==============================] - 2s 224us/step - loss: 0.4171 - acc: 0.8717
Epoch 5/50
10000/10000 [==============================] - 2s 234us/step - loss: 0.3808 - acc: 0.8801
Epoch 6/50
10000/10000 [==============================] - 2s 216us/step - loss: 0.3547 - acc: 0.8901
Epoch 7/50
10000/10000 [==============================] - 2s 239us/step - loss: 0.3352 - acc: 0.8976
Epoch 8/50
10000/10000 [==============================] - 3s 251us/step - loss: 0.3070 - acc: 0.9068
Epoch 9/50
10000/10000 [==============================] - 2s 229us/step - loss: 0.2920 - acc: 0.9111
Epoch 10/50
10000/10000 [==================

## 分類

In [50]:
# テストデータで推測結果を出力
cnn_pred = model.predict(X_test)

In [51]:
print(cnn_pred[0])

[1.3158771e-08 8.4456736e-10 1.9096547e-05 2.2043609e-04 8.4381586e-09
 6.0182145e-07 3.6360989e-10 4.5563708e-09 9.9975973e-01 1.1844171e-07]


In [52]:
# 推測結果をクラスラベルへ変換
y_pred_CNN = np.argmax(cnn_pred, axis=1)
y_test_CNN = np.argmax(y_test, axis=1)

In [53]:
print(confusion_matrix(y_test_CNN,y_pred_CNN))

[[ 969    0    4    0    0    0    5    1    2    2]
 [   0 1132   11    0    1    0    1    2    4    1]
 [   3    7  940    0    2    0    1    3   10    1]
 [   2    3   18  980    2    4    1    4   10   10]
 [   2    0    3    0  864    0    9    2    1   25]
 [   4    1    1   18    1  896    7    0    6    3]
 [   4    2    0    0    1    3  949    0    2    0]
 [   2    3   13    1    6    1    0 1011    1   17]
 [   5    5   11    9    4    0    4    4  920    7]
 [   5    5    1    9    5    0    0    5    4 1002]]


In [54]:
print(accuracy_score(y_test_CNN,y_pred_CNN))

0.9663
